<a href="https://colab.research.google.com/github/raisinghanii/Movie-Recommendation/blob/master/SVD_on_ml_latest_small_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-small.zip
!unzip ml-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0  2269k      0 --:--:-- --:--:-- --:--:-- 2263k
Archive:  ml-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [ ]:
!curl http://files.grouplens.org/datasets/movielens/ml-latest.zip -o movie.zip
!unzip movie.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  264M  100  264M    0     0   103M      0  0:00:02  0:00:02 --:--:--  102M
Archive:  movie.zip
replace ml-latest/links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ml-latest/tags.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ml-latest/genome-tags.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ml-latest/ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ml-latest/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ml-latest/genome-scores.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ml-latest/movies.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_ratings = pd.read_csv('ml-latest/ratings.csv')
df_movies = pd.read_csv('ml-latest/movies.csv')

In [ ]:
ratings.head()

userId  movieId  rating   timestamp
0       1      307     3.5  1256677221
1       1      481     3.5  1256677456
2       1     1091     1.5  1256677471
3       1     1257     4.5  1256677460
4       1     1449     4.5  1256677264

In [ ]:
movies.head()

movieId  ...                                       genres
0        1  ...  Adventure|Animation|Children|Comedy|Fantasy
1        2  ...                   Adventure|Children|Fantasy
2        3  ...                               Comedy|Romance
3        4  ...                         Comedy|Drama|Romance
4        5  ...                                       Comedy

[5 rows x 3 columns]

In [ ]:
num_users = len(df_ratings.userId.unique())
num_items = len(df_ratings.movieId.unique())
print('There are {} unique users and {} unique movies in this data set'.format(num_users, num_items))

There are 283228 unique users and 53889 unique movies in this data set


In [ ]:
# count of each rating
df_ratings_cnt_tmp = pd.DataFrame(df_ratings.groupby('rating').size(), columns=['count'])
df_ratings_cnt_tmp

count
rating         
0.5      442388
1.0      886233
1.5      441354
2.0     1850627
2.5     1373419
3.0     5515668
3.5     3404360
4.0     7394710
4.5     2373550
5.0     4071135

In [ ]:
# there are a lot more counts in rating of zero
total_cnt = num_users * num_items
rating_zero_cnt = total_cnt - df_ratings.shape[0]
# append counts of zero rating to df_ratings_cnt
df_ratings_cnt = df_ratings_cnt_tmp.append(
    pd.DataFrame({'count': rating_zero_cnt}, index=[0.0]),
    verify_integrity=True,
).sort_index()
df_ratings_cnt

count
0.0  15235120248
0.5       442388
1.0       886233
1.5       441354
2.0      1850627
2.5      1373419
3.0      5515668
3.5      3404360
4.0      7394710
4.5      2373550
5.0      4071135

In [ ]:
# add log count
df_ratings_cnt['log_count'] = np.log(df_ratings_cnt['count'])
df_ratings_cnt

count  log_count
0.0  15235120248  23.446869
0.5       442388  12.999943
1.0       886233  13.694735
1.5       441354  12.997603
2.0      1850627  14.431035
2.5      1373419  14.132814
3.0      5515668  15.523103
3.5      3404360  15.040568
4.0      7394710  15.816275
4.5      2373550  14.679897
5.0      4071135  15.219432

In [ ]:
# get rating frequency
df_movies_cnt = pd.DataFrame(df_ratings.groupby('movieId').size(), columns=['count'])
df_movies_cnt.head()

count
movieId       
1        68469
2        27143
3        15585
4         2989
5        15474

In [ ]:
df_movies_cnt['count'].quantile(np.arange(1, 0.6, -0.05))

1.00    97999.0
0.95     1855.0
0.90      531.0
0.85      205.0
0.80       91.0
0.75       48.0
0.70       28.0
0.65       18.0
Name: count, dtype: float64

In [ ]:
# filter data
popularity_thres = 50
popular_movies = list(set(df_movies_cnt.query('count >= @popularity_thres').index))
df_ratings = pd.merge(df_movies,df_ratings)
df_ratings_drop_movies = df_ratings[df_ratings.movieId.isin(popular_movies)]
print('shape of original ratings data: ', df_ratings.shape)
print('shape of ratings data after dropping unpopular movies: ', df_ratings_drop_movies.shape)

shape of original ratings data:  (27753444, 6)
shape of ratings data after dropping unpopular movies:  (27430748, 6)


In [ ]:
# get number of ratings given by every user
df_users_cnt = pd.DataFrame(df_ratings_drop_movies.groupby('userId').size(), columns=['count'])
df_users_cnt.head()

count
userId       
1          16
2          15
3          11
4         736
5          72

In [ ]:
df_users_cnt['count'].quantile(np.arange(1, 0.5, -0.05))

1.00    9384.0
0.95     403.0
0.90     239.0
0.85     164.0
0.80     121.0
0.75      94.0
0.70      73.0
0.65      58.0
0.60      47.0
0.55      37.0
Name: count, dtype: float64

In [ ]:
# filter data
ratings_thres = 50
active_users = list(set(df_users_cnt.query('count >= @ratings_thres').index))
df_ratings_drop_users = df_ratings_drop_movies[df_ratings_drop_movies.userId.isin(active_users)]
print('shape of original ratings data: ', df_ratings.shape)
print('shape of ratings data after dropping both unpopular movies and inactive users: ', df_ratings_drop_users.shape)

shape of original ratings data:  (27753444, 6)
shape of ratings data after dropping both unpopular movies and inactive users:  (24178982, 6)


In [ ]:
dropped = df_ratings_drop_users.drop(['timestamp','genres'],axis=1)
dropped

movieId             title  userId  rating
0               1  Toy Story (1995)       4     4.0
1               1  Toy Story (1995)      10     5.0
2               1  Toy Story (1995)      14     4.5
3               1  Toy Story (1995)      15     4.0
7               1  Toy Story (1995)      31     5.0
...           ...               ...     ...     ...
27752464   192307    The Meg (2018)  270572     3.0
27752465   192307    The Meg (2018)  272459     5.0
27752466   192307    The Meg (2018)  275841     1.5
27752467   192307    The Meg (2018)  279522     3.0
27752468   192307    The Meg (2018)  279713     3.0

[24178982 rows x 4 columns]

In [ ]:
movielens_df = dropped[['userId','title','rating']]
movielens_df

userId             title  rating
0              4  Toy Story (1995)     4.0
1             10  Toy Story (1995)     5.0
2             14  Toy Story (1995)     4.5
3             15  Toy Story (1995)     4.0
7             31  Toy Story (1995)     5.0
...          ...               ...     ...
27752464  270572    The Meg (2018)     3.0
27752465  272459    The Meg (2018)     5.0
27752466  275841    The Meg (2018)     1.5
27752467  279522    The Meg (2018)     3.0
27752468  279713    The Meg (2018)     3.0

[24178982 rows x 3 columns]

In [ ]:
merged = pd.merge(ratings,movies)
merged

NameError: ignored

In [ ]:
merged = merged.drop(['timestamp','genres','movieId'],axis=1)
merged

userId  rating                             title
0            1     4.0                  Toy Story (1995)
1            5     4.0                  Toy Story (1995)
2            7     4.5                  Toy Story (1995)
3           15     2.5                  Toy Story (1995)
4           17     4.5                  Toy Story (1995)
...        ...     ...                               ...
100831     610     2.5                  Bloodmoon (1997)
100832     610     4.5  Sympathy for the Underdog (1971)
100833     610     3.0                     Hazard (2005)
100834     610     3.5                Blair Witch (2016)
100835     610     3.5                         31 (2016)

[100836 rows x 3 columns]

In [ ]:
movielens_df = merged[['userId','title','rating']]
movielens_df

NameError: ignored

In [ ]:
#mast function hai..

def get_title_from_index(index):
    return movies[movies.movieId == index]["title"].values[0]

def get_index_from_title(title):
    return movies[movies.title == title]["movieId"].values[0]

In [ ]:
!pip install surprise

In [ ]:
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate, train_test_split


reader = Reader(rating_scale=(1, 5))

In [ ]:
movielens_df = ratings.drop(['timestamp'],axis=1)
movielens_df

userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
...        ...      ...     ...
100831     610   166534     4.0
100832     610   168248     5.0
100833     610   168250     5.0
100834     610   168252     5.0
100835     610   170875     3.0

[100836 rows x 3 columns]

In [ ]:
data = Dataset.load_from_df(movielens_df, reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=.01)

In [ ]:
model = SVD(n_factors=100)
model.fit(trainset)

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pickle
model = pickle.load(open('drive/My Drive/model_svd_100','rb'))
model

In [ ]:
predictions = model.test(testset)

from surprise import accuracy

accuracy.rmse(predictions)

In [ ]:
from surprise import accuracy

predictions = model.test(testset)


In [ ]:
import pickle
with open('model_svd_100','wb') as f:
  pickle.dump(model,f)

In [ ]:
dbfile = open('model_knn_51', 'rb')      
model_51 = pickle.load(dbfile) 
  
model_51.qi.shape

ERROR! Session/line number was not unique in database. History logging moved to new session 59


(13357, 51)

In [ ]:
pd.DataFrame(model_51.qi).iloc[0].pow(2).sum()
model_51.qi /= np.linalg.norm(model_51.qi, ord=2, axis=1).reshape(-1, 1)
pd.DataFrame(model_51.qi).iloc[0].pow(2).sum()

1.0000000000000002

In [ ]:
pd.DataFrame(model.qi).iloc[0].pow(2).sum()
model.qi /= np.linalg.norm(model.qi, ord=2, axis=1).reshape(-1, 1)
pd.DataFrame(model.qi).iloc[0].pow(2).sum()

1.0

In [ ]:
model.qi.shape

(13357, 100)

In [ ]:
def display(similarity_table):
    similarity_table = pd.DataFrame(
        similarity_table,
        columns=['vector cosine distance', 'movie title']
    ).sort_values('vector cosine distance', ascending=True)
    return similarity_table.iloc[:20]

In [ ]:
from scipy.spatial.distance import cosine


def get_vector_by_movie_title(movie_title: str, trained_model: SVD) -> np.array:
    """Returns the latent features of a movie in the form of a numpy array"""
    movie_row_idx = trainset._raw2inner_id_items[movie_title]
    #movie_row_idx = trained_model.qi[movie_title]
    return trained_model.qi[movie_row_idx]


def cosine_distance(vector_a: np.array, vector_b: np.array) -> float:
    """Returns a float indicating the similarity between two vectors"""
    return cosine(vector_a, vector_b)

In [ ]:
def get_top_similarities(movie_title: str, model: SVD) -> pd.DataFrame:
    """Returns the top 5 most similar movies to a specified movie
    
    This function iterates over every possible movie in MovieLens and calculates
    distance between `movie_title` vector and that movie's vector.
    """
    
    # Get the first movie vector
    movie_vector: np.array = get_vector_by_movie_title(movie_title, model)
    print("latent features ", movie_vector)
    similarity_table = []
    
    # Iterate over every possible movie and calculate similarity
    for other_movie_title in model.trainset._raw2inner_id_items.keys():
        other_movie_vector = get_vector_by_movie_title(other_movie_title, model)
        
        # Get the second movie vector, and calculate distance
        similarity_score = cosine_distance(other_movie_vector, movie_vector)
        similarity_table.append((similarity_score,other_movie_title) )
    
    # sort movies by ascending similarity
    return display(sorted(similarity_table))


In [ ]:
get_top_similarities('Avengers: Age of Ultron (2015)',model)

latent features  [ 0.15353463 -0.02727099  0.11341589  0.02315515  0.04158798 -0.15576676
 -0.00081905 -0.10214993 -0.16330124  0.0211672   0.17277914 -0.00046341
  0.13163803  0.09979592  0.08185749  0.03604156  0.14783946  0.0305962
  0.05466346 -0.15186023 -0.0961948   0.11167909  0.16420613 -0.0181715
  0.0504604  -0.06170065  0.23778008  0.05628853  0.12789797  0.00925161
  0.03192512  0.06490928 -0.07562076  0.15892227  0.08738778  0.06458731
  0.08499189  0.0529424  -0.0239857   0.05811249  0.02734578 -0.00252232
  0.11544705  0.05168708 -0.13617273  0.11198784 -0.17592121 -0.02390081
 -0.09244364  0.02798003 -0.05428188  0.19470477 -0.07744764  0.00946
  0.05268605 -0.27947872  0.02862078  0.03058556  0.0393725   0.0040305
 -0.15520832 -0.06144624 -0.02091177 -0.05410699  0.07416543 -0.07286184
  0.02455893 -0.11925035  0.0476023   0.12598021 -0.01162    -0.04162715
  0.02067518  0.02397816 -0.12595411 -0.01977485 -0.14025787  0.04957602
 -0.10403716  0.06687836  0.15534029  0.

vector cosine distance                                 movie title
0                 0.000000              Avengers: Age of Ultron (2015)
1                 0.079782                        Avengers, The (2012)
2                 0.102101           Captain America: Civil War (2016)
3                 0.108724   Captain America: The First Avenger (2011)
4                 0.111744                           Iron Man 3 (2013)
5                 0.114469                           Iron Man 2 (2010)
6                 0.120904                                 Thor (2011)
7                 0.121400  Captain America: The Winter Soldier (2014)
8                 0.121662                 Thor: The Dark World (2013)
9                 0.161081                       Doctor Strange (2016)
10                0.181169                              Ant-Man (2015)
11                0.191242                             Iron Man (2008)
12                0.243397            Guardians of the Galaxy 2 (2017)
13                0.250868      Avengers: Infinity War - Part I (2018)
14                0.251707           Untitled Spider-Man Reboot (2017)
15                0.252725                       Thor: Ragnarok (2017)
16                0.258568              Guardians of the Galaxy (2014)
17                0.279841                    X-Men: Apocalypse (2016)
18                0.302348           X-Men: Days of Future Past (2014)
19                0.302444                 Ant-Man and the Wasp (2018)

In [ ]:
get_top_similarities('Iron Man (2008)',model_51)

latent features  [ 0.07054877 -0.07240716  0.1814819   0.16233063  0.05739997 -0.04114154
  0.01480378 -0.33317211  0.18243732  0.10795722 -0.01969193 -0.1256602
  0.15654159 -0.18672918  0.05043408  0.3138876   0.01691325 -0.0020539
  0.04269171 -0.18706786  0.1040003  -0.01439879 -0.24333908  0.29755093
  0.09278643  0.03494333 -0.03082303  0.07022575  0.26580731 -0.05568917
 -0.14092345 -0.03185636 -0.14226451  0.00185027  0.01957412  0.08628148
  0.0719336  -0.00844865  0.04378799  0.00887385 -0.21107674  0.05417043
 -0.0585013   0.06717716  0.12244464 -0.06923897  0.37508992  0.01094342
 -0.01348267  0.181892    0.04360788]


vector cosine distance                                 movie title
0                 0.000000                             Iron Man (2008)
1                 0.074038                        Avengers, The (2012)
2                 0.138986                           Iron Man 2 (2010)
3                 0.146484              Avengers: Age of Ultron (2015)
4                 0.148424  Captain America: The Winter Soldier (2014)
5                 0.152231                           Iron Man 3 (2013)
6                 0.158817                                 Thor (2011)
7                 0.159390   Captain America: The First Avenger (2011)
8                 0.160617           Captain America: Civil War (2016)
9                 0.165799                   X-Men: First Class (2011)
10                0.188445                              Ant-Man (2015)
11                0.207791           X-Men: Days of Future Past (2014)
12                0.223439                                X-Men (2000)
13                0.224195              Guardians of the Galaxy (2014)
14                0.227713                 Thor: The Dark World (2013)
15                0.265978           Untitled Spider-Man Reboot (2017)
16                0.275150      Avengers: Infinity War - Part I (2018)
17                0.283446                     X2: X-Men United (2003)
18                0.283573                            Star Trek (2009)
19                0.290260                       Doctor Strange (2016)

In [ ]:
get_top_similarities('3 Idiots (2009)',model)

latent features  [-0.11483295 -0.04775186  0.10419151  0.05305675 -0.03323165  0.115288
  0.0937025   0.11619754 -0.03474302 -0.01780333 -0.03352075 -0.11291951
  0.06568764 -0.17149854  0.07531909  0.02625847  0.05211318 -0.22787218
  0.00806377  0.01025634  0.10356632 -0.01635829  0.02485059  0.03601164
 -0.12621525  0.03474541 -0.05334358 -0.19381952  0.04610365 -0.07839868
 -0.05086207  0.10309622 -0.02017711  0.11521759  0.09477878  0.02471481
  0.1123164   0.01006954  0.07176244  0.0298872   0.14880472 -0.03276587
  0.10657186 -0.08719321 -0.13962806  0.26014401 -0.05831848 -0.19587489
 -0.14603664 -0.03642177 -0.08869648 -0.05941467  0.02345279  0.06290936
 -0.13048959  0.05947867 -0.0268932  -0.08014755 -0.0626099   0.08758177
 -0.21097354 -0.19295781 -0.08271043 -0.06770179 -0.05736309 -0.06447391
  0.13185218 -0.01427198  0.02250041  0.12498955 -0.11376357 -0.13752587
  0.0596392  -0.09196162  0.13373583  0.04508116  0.09764906 -0.02286552
 -0.10824537  0.02010839  0.04853737

vector cosine distance                                    movie title
0                 0.000000                                3 Idiots (2009)
1                 0.604221                                Drishyam (2015)
2                 0.625719  As it is in Heaven (Så som i himmelen) (2004)
3                 0.627103                    Broadway Melody, The (1929)
4                 0.632583                            Saving Grace (2000)
5                 0.635714                             Get Over It (2001)
6                 0.643419                                  Barfi! (2012)
7                 0.644966                   Long, Hot Summer, The (1958)
8                 0.649709                                 Ghajini (2008)
9                 0.653272    Swades: We, the People (Our Country) (2004)
10                0.655610                          OMG Oh My God! (2012)
11                0.658144                           Paleface, The (1948)
12                0.660027                    The Spiral Staircase (1945)
13                0.660293                    Bandit, The (Eskiya) (1996)
14                0.660830                        Festival Express (2003)
15                0.661517                      Red Cliff (Chi bi) (2008)
16                0.662811  Like Stars on Earth (Taare Zameen Par) (2007)
17                0.664031                                      PK (2014)
18                0.664352    Eat Drink Man Woman (Yin shi nan nu) (1994)
19                0.666043                            Mrs. Miniver (1942)

In [ ]:
get_top_similarities('Gangs of Wasseypur (2012)',model)

latent features  [ 7.10894527e-02  2.82009864e-02 -8.85498784e-02 -2.61779398e-02
  3.80171137e-02  7.27939544e-02 -1.16410196e-01  1.10412633e-01
  8.10831934e-02  5.33125201e-03 -8.57898934e-02  3.77817512e-02
  1.00592571e-01 -9.62263275e-02 -1.09663629e-01  9.69421098e-02
  3.38538902e-02 -2.17957017e-01  7.70482258e-02 -9.54452616e-02
  1.57087788e-02 -3.25613323e-02 -8.20258865e-02 -1.25595730e-01
  1.14175148e-02 -7.96080098e-02 -1.56360125e-01 -4.92922128e-02
  2.94024422e-02 -2.94895423e-02 -2.20103511e-03  3.29505444e-02
  1.23769651e-01 -2.13500233e-01  1.25371472e-01  1.04038608e-01
  1.50059412e-02  4.50837561e-02  1.26740591e-01  1.05852205e-01
  6.52590814e-02  1.30260786e-01 -1.31518862e-01 -1.44385229e-01
  1.39504006e-01  1.05735306e-01 -2.62665408e-02  4.69014320e-02
 -3.60680356e-02  1.73294958e-01  6.78295017e-02 -8.11472302e-02
  2.63787798e-02 -6.07419598e-02 -2.22980175e-01 -3.03771261e-02
  1.21570351e-01  1.16417640e-01  1.24622297e-01  9.12443118e-02
 -9.2979

vector cosine distance                                        movie title
0                 0.000000                          Gangs of Wasseypur (2012)
1                 0.450781                                       Heima (2007)
2                 0.459951      Castaway on the Moon (Kimssi pyoryugi) (2009)
3                 0.480313  Moment to Remember, A (Nae meorisokui jiwoogae...
4                 0.490239                    Head-On (Gegen die Wand) (2004)
5                 0.504052  Battle of Algiers, The (La battaglia di Algeri...
6                 0.506601                                           Z (1969)
7                 0.517781                                Hustler, The (1961)
8                 0.529842                                     Yojimbo (1961)
9                 0.535269         Claymation Christmas Celebration, A (1987)
10                0.536062                         Steamboat Bill, Jr. (1928)
11                0.536628                City of God (Cidade de Deus) (2002)
12                0.539148                   Sanjuro (Tsubaki Sanjûrô) (1962)
13                0.540306        Days of Being Wild (A Fei jingjyuhn) (1990)
14                0.540503                             No End in Sight (2007)
15                0.543401                                   Kagemusha (1980)
16                0.544579     Grave of the Fireflies (Hotaru no haka) (1988)
17                0.545146                                         Ran (1985)
18                0.548621                         Rashomon (Rashômon) (1950)
19                0.552803        Seven Samurai (Shichinin no samurai) (1954)

In [ ]:
get_top_similarities('Iron Man (2008)', model)

latent features  [ 0.17552907 -0.18813341  0.12201248  0.09287365  0.07007866 -0.13677
 -0.00353993 -0.01518915 -0.16066353  0.07955286  0.08910807 -0.01536206
  0.06510739  0.12646873  0.07920819 -0.03657941  0.20425814 -0.02365248
  0.10615341 -0.16920066 -0.15979504  0.13779279  0.14462988  0.00546766
  0.07330255 -0.06181985  0.19620994  0.00273506  0.19937249  0.06423357
  0.06168199  0.16440559 -0.23798336  0.08280221  0.03408315 -0.07836399
  0.06376274  0.08172686  0.00563531  0.03786434  0.03274485  0.09338517
  0.12804572  0.0739328  -0.0572389   0.05587276 -0.11645052  0.05282576
 -0.01934389 -0.10031054 -0.03549819  0.03436164 -0.12203049  0.02920926
  0.03031544 -0.23881316  0.04965412 -0.0360773   0.04980412 -0.04820362
 -0.13944348 -0.0312933   0.08414125  0.0186998   0.02139751  0.04021273
  0.05901124 -0.07967613  0.09653154  0.12407959  0.01884746 -0.02261489
 -0.01915884  0.02666894 -0.10639467 -0.05156218 -0.04735589  0.05560504
 -0.18436124  0.07491417  0.09747252 

vector cosine distance                                 movie title
0                 0.000000                             Iron Man (2008)
1                 0.102859                        Avengers, The (2012)
2                 0.151409                           Iron Man 2 (2010)
3                 0.165868                           Iron Man 3 (2013)
4                 0.191242              Avengers: Age of Ultron (2015)
5                 0.200615  Captain America: The Winter Soldier (2014)
6                 0.201220   Captain America: The First Avenger (2011)
7                 0.208038           Captain America: Civil War (2016)
8                 0.218950                                 Thor (2011)
9                 0.236538                   X-Men: First Class (2011)
10                0.256729              Guardians of the Galaxy (2014)
11                0.259841                              Ant-Man (2015)
12                0.270804                 Thor: The Dark World (2013)
13                0.281870           X-Men: Days of Future Past (2014)
14                0.288823                                X-Men (2000)
15                0.303069                        Batman Begins (2005)
16                0.331225                     X2: X-Men United (2003)
17                0.334338                 Incredible Hulk, The (2008)
18                0.336676                       Doctor Strange (2016)
19                0.341866              Amazing Spider-Man, The (2012)

In [ ]:
get_top_similarities('Pulp Fiction (1994)',model)

latent features  [-0.04839189 -0.1844158  -0.13890106  0.13925739  0.09967151  0.13303166
  0.11919993 -0.24798814  0.17622715 -0.04009086  0.14440954 -0.03109975
 -0.08194974 -0.05421892  0.02274573  0.06118848 -0.04848741  0.06308001
  0.14271108 -0.02636363  0.07318896 -0.16887392 -0.09766078 -0.12173491
 -0.00614876 -0.10517834 -0.0081462  -0.05817587 -0.04535443  0.06244435
 -0.15012231 -0.01921661  0.01684323 -0.15794522 -0.03924464 -0.07838745
  0.09534631  0.00952775  0.03560992 -0.02784055  0.11580837  0.06046082
  0.01345294 -0.16400943  0.13982837  0.24746196  0.02801263  0.13249952
 -0.06086004  0.06643681  0.02133916 -0.05479529 -0.06957795  0.01197666
 -0.03426902  0.12143498  0.00346926  0.11516774  0.16637652  0.00330139
  0.07785919  0.14538263  0.03175404  0.01541553 -0.08189921  0.02908646
  0.03795118  0.05566053  0.01095022 -0.04828628  0.09271469 -0.06841319
 -0.22382488  0.00739233  0.07808574 -0.06255621 -0.00916604  0.05909066
 -0.03395964  0.12671141  0.002748

vector cosine distance                    movie title
0                 0.000000            Pulp Fiction (1994)
1                 0.091905          Reservoir Dogs (1992)
2                 0.254497           Trainspotting (1996)
3                 0.321900              Fight Club (1999)
4                 0.339153              Goodfellas (1990)
5                 0.341943            True Romance (1993)
6                 0.345212       Full Metal Jacket (1987)
7                 0.365831     Usual Suspects, The (1995)
8                 0.370384         American Beauty (1999)
9                 0.371745       Kill Bill: Vol. 1 (2003)
10                0.380707       Kill Bill: Vol. 2 (2004)
11                0.381357     Clockwork Orange, A (1971)
12                0.381997                Sin City (2005)
13                0.396216    Inglourious Basterds (2009)
14                0.402962           Boogie Nights (1997)
15                0.403710             Taxi Driver (1976)
16                0.407064                 Memento (2000)
17                0.408469  No Country for Old Men (2007)
18                0.410030                  Casino (1995)
19                0.423631    Seven (a.k.a. Se7en) (1995)

In [ ]:
get_index_from_title('Dark Knight, The (2008)')

58559

In [ ]:
get_top_similarities('Dark Knight, The (2008)',model_51)

latent features  [-0.01161933 -0.05719063  0.16620988  0.169979   -0.11550246 -0.28770528
  0.08509522 -0.23611153 -0.01740417  0.03782202  0.10959899 -0.21454243
  0.05592581 -0.03271871  0.20867309  0.42789291  0.06567864 -0.15038012
  0.16760153 -0.05764374  0.02174444  0.0537782  -0.13281617  0.16626882
  0.17833632 -0.01646064 -0.06661313  0.06221696  0.18775435 -0.1028663
 -0.16983898  0.12718509 -0.00580128 -0.02835694 -0.04723784 -0.02131642
 -0.05462247 -0.12796164 -0.08489367 -0.01933409 -0.01551241  0.09975913
  0.18573774  0.2252033   0.06457234 -0.14112912  0.03208485 -0.22394532
  0.0604465   0.21747391 -0.00683003]


vector cosine distance                    movie title
0                 0.000000        Dark Knight, The (2008)
1                 0.081091           Batman Begins (2005)
2                 0.099311  Dark Knight Rises, The (2012)
3                 0.341070               Inception (2010)
4                 0.350319           Departed, The (2006)
5                 0.397912                Iron Man (2008)
6                 0.436827           Prestige, The (2006)
7                 0.445908              Fight Club (1999)
8                 0.446941               Gladiator (2000)
9                 0.448195           Casino Royale (2006)
10                0.465871                     300 (2007)
11                0.486647              Spider-Man (2002)
12                0.493433                Kick-Ass (2010)
13                0.503880                Sin City (2005)
14                0.510184             Matrix, The (1999)
15                0.512307      X-Men: First Class (2011)
16                0.523291        Band of Brothers (2001)
17                0.530552                   Logan (2017)
18                0.530846           Avengers, The (2012)
19                0.533273            Interstellar (2014)

In [ ]:
get_top_similarities('Dark Knight, The (2008)',model)

latent features  [ 0.18584329 -0.28451003  0.10683197  0.0734247  -0.10122333 -0.06841313
 -0.00978157 -0.07500063 -0.06663965  0.02640295  0.1345997   0.02701861
 -0.15397375  0.09252397  0.02860629  0.18791138  0.15446019  0.04389735
  0.10891514 -0.12482751 -0.14881518  0.00059787  0.14593638 -0.00999155
 -0.01328814 -0.11703168  0.10228948  0.07244124  0.21665683  0.04404316
  0.04149232  0.08196312 -0.09939727 -0.01941636  0.03294283 -0.17127132
  0.04688111  0.22350605  0.0061191  -0.00730855  0.04238877  0.10595122
  0.09249429  0.0643432  -0.03921778  0.16983702  0.055571    0.06645806
 -0.07941463  0.11082217 -0.08258824 -0.16749303 -0.10446101  0.02799695
 -0.11084733 -0.05942003 -0.0488213   0.02305625  0.07126456 -0.00763064
 -0.07383996  0.02771389 -0.02094991  0.14415814 -0.07472137  0.01318692
 -0.05953961 -0.05568221  0.05631682 -0.05480491  0.16715506  0.0188058
 -0.05288435 -0.15043706 -0.10185502 -0.13161945  0.05819289  0.01063689
 -0.06992476  0.04113804  0.0476908

vector cosine distance                             movie title
0                 0.000000                 Dark Knight, The (2008)
1                 0.107406           Dark Knight Rises, The (2012)
2                 0.116137                    Batman Begins (2005)
3                 0.347603                        Inception (2010)
4                 0.398045                    Departed, The (2006)
5                 0.417526                         Iron Man (2008)
6                 0.444655               X-Men: First Class (2011)
7                 0.469233                        Gladiator (2000)
8                 0.472330                    Prestige, The (2006)
9                 0.486254       X-Men: Days of Future Past (2014)
10                0.499904                    Avengers, The (2012)
11                0.508992                       Fight Club (1999)
12                0.510178                     Interstellar (2014)
13                0.512339                            Logan (2017)
14                0.519582  Avengers: Infinity War - Part I (2018)
15                0.530867                              300 (2007)
16                0.535308       Captain America: Civil War (2016)
17                0.546979                    Casino Royale (2006)
18                0.556456                          Warrior (2011)
19                0.559026             Incredible Hulk, The (2008)

In [ ]:
get_index_from_title('Terminator, The (1984)')

1240

In [ ]:
get_top_similarities('Terminator, The (1984)',model)

latent features  [-4.21920212e-02 -1.42539930e-01  3.67478960e-02  3.86570884e-02
  5.99149147e-02 -3.76251286e-02  6.25728065e-02 -1.41886381e-01
  2.33768776e-03  1.69825249e-01 -4.96612012e-05  3.12746653e-02
  4.44438213e-02  6.16832699e-02 -7.06869364e-02 -3.73112650e-02
 -7.32005540e-03  6.74981323e-02  1.59145975e-01 -1.58470092e-02
  2.20680234e-02  3.42427758e-03 -8.98824772e-02  1.90646261e-01
  1.72475564e-01 -4.84907549e-02  1.61605415e-01 -4.01272914e-02
 -3.20208548e-03  1.23533486e-01 -1.39960235e-02  7.46859035e-02
  2.28939795e-03 -1.46911517e-01 -9.58854915e-03 -3.36125239e-01
 -1.44204576e-02  6.74290035e-02 -1.40141425e-04 -1.91464698e-01
 -8.54880935e-02  2.65515742e-02 -8.80993334e-02 -5.87795778e-03
 -5.39030106e-03  3.32663095e-02 -1.47604732e-01  9.17482710e-02
  1.40257908e-01 -1.02721490e-01  1.72226050e-01 -8.14615293e-02
  9.21181049e-02 -1.44053461e-01 -1.00876067e-01  2.31698915e-01
  8.42111028e-02  1.22849163e-01  9.08458612e-02  1.26018474e-02
  1.1117

vector cosine distance                                        movie title
0                 0.000000                             Terminator, The (1984)
1                 0.092415                  Terminator 2: Judgment Day (1991)
2                 0.192534                                      Aliens (1986)
3                 0.221587                                    Predator (1987)
4                 0.226520                                       Alien (1979)
5                 0.245486                                Total Recall (1990)
6                 0.269122                                    Die Hard (1988)
7                 0.333571                                     RoboCop (1987)
8                 0.443601               Road Warrior, The (Mad Max 2) (1981)
9                 0.448048                                Blade Runner (1982)
10                0.455024                                        Jaws (1975)
11                0.455674                                     Tremors (1990)
12                0.458472  Raiders of the Lost Ark (Indiana Jones and the...
13                0.458636                        Escape from New York (1981)
14                0.466192                                   True Lies (1994)
15                0.484758            First Blood (Rambo: First Blood) (1982)
16                0.489377                                  Thing, The (1982)
17                0.492379                                    Fly, The (1986)
18                0.496910          Terminator 3: Rise of the Machines (2003)
19                0.499370                               Jurassic Park (1993)

In [ ]:
get_index_from_title('Inception (2010)')

79132

In [ ]:
get_top_similarities(79132,model)

latent features  [ 0.22508037  0.0761223  -0.00557528 -0.17448671  0.28154082  0.10353134
  0.05982008 -0.08692264 -0.07934035 -0.10926932 -0.14299321 -0.07295278
 -0.1481284   0.11003824 -0.06683048  0.0126789  -0.11807297 -0.23694496
 -0.06338461 -0.12602155 -0.07462489 -0.00554455 -0.18211114  0.22328154
  0.0663815  -0.16592312  0.05701576  0.19826746  0.45038244  0.10038063
 -0.02084792 -0.12331163 -0.03578924  0.12861898  0.14206742 -0.18772506
 -0.18792364 -0.04083318  0.01491454  0.06935497 -0.06637608  0.06526788
  0.21411546  0.0409536  -0.07647835  0.12414605  0.08057078  0.12380207
  0.00954568 -0.1568797   0.09151138]


vector cosine distance                          movie title
0                 0.000000                     Inception (2010)
1                 0.513139            Boniface's Holiday (1965)
2                 0.521392                Reservoir Dogs (1992)
3                 0.544455                        Snatch (2000)
4                 0.546778                     Entourage (2015)
5                 0.548053                   On the Town (1949)
6                 0.561484    10 Things I Hate About You (1999)
7                 0.564720                 Royal Wedding (1951)
8                 0.580238                          Exam (2009)
9                 0.589876                   Bring It On (2000)
10                0.590085                           JFK (1991)
11                0.594095          Herbie: Fully Loaded (2005)
12                0.595599                 Danny Collins (2015)
13                0.595767                       Airport (1970)
14                0.604979                          Eros (2004)
15                0.609797  Ice Age: A Mammoth Christmas (2011)
16                0.611662                    Straw Dogs (1971)
17                0.613250                  Taking Lives (2004)
18                0.616255         Kicking and Screaming (1995)
19                0.616312          Beginning of the End (1957)

In [ ]:
get_index_from_title('American Pie (1999)')

2706

In [ ]:
get_top_similarities('American Pie (1999)',model)

latent features  [ 0.06916975 -0.17691849 -0.12345927  0.02577529  0.00414617 -0.1580612
 -0.00589978  0.02984275  0.0267413   0.11437303 -0.06322218 -0.17900712
  0.02497988 -0.01365376  0.05519439 -0.04615944  0.00375662  0.12910838
  0.09163582 -0.01957876  0.11676284 -0.05718277 -0.02862672 -0.11794136
  0.20003585 -0.10755292 -0.0469452  -0.01876623  0.06642592  0.04358813
 -0.04716921  0.15104787 -0.02429854 -0.01607412 -0.14765567  0.02068257
  0.06512374 -0.06832164  0.04454182 -0.14209621  0.09977411  0.06985124
  0.15959499 -0.06914175 -0.11014516  0.03012536 -0.07178944  0.08466275
 -0.13633665 -0.06402207  0.00837308 -0.04429217 -0.19193552  0.07177527
 -0.01410122 -0.08287763 -0.02696779  0.08201471  0.16834248  0.17393747
  0.03228317  0.12616427 -0.07678823 -0.00694182 -0.1664792  -0.0275242
  0.20084229  0.09704683 -0.04966189  0.08543016 -0.00900754 -0.23977182
  0.02099022 -0.02402234  0.01556488 -0.06644158  0.0408411   0.16337986
 -0.02912587  0.02242136 -0.17413148

vector cosine distance                                   movie title
0                 0.000000                           American Pie (1999)
1                 0.089224                         American Pie 2 (2001)
2                 0.214245                              Road Trip (2000)
3                 0.280121      American Wedding (American Pie 3) (2003)
4                 0.280123           There's Something About Mary (1998)
5                 0.364744                          Hangover, The (2009)
6                 0.373616                       Wedding Crashers (2005)
7                 0.381767      American Reunion (American Pie 4) (2012)
8                 0.389868                            Scary Movie (2000)
9                 0.403987                40-Year-Old Virgin, The (2005)
10                0.415994                       Meet the Parents (2000)
11                0.423353                  40 Days and 40 Nights (2002)
12                0.429703                             Old School (2003)
13                0.448530           Fast Times at Ridgemont High (1982)
14                0.460242                          Varsity Blues (1999)
15                0.460654                                Porky's (1982)
16                0.461179                 Not Another Teen Movie (2001)
17                0.462002                               EuroTrip (2004)
18                0.464945  Austin Powers: The Spy Who Shagged Me (1999)
19                0.467477                    Girl Next Door, The (2004)

In [ ]:
get_top_similarities('American Pie (1999)',model_51)

latent features  [ 0.02160641 -0.00337964  0.05465491  0.01870211 -0.02229971 -0.08920482
  0.03775913 -0.02775662  0.08817178  0.23162083 -0.09419947 -0.05844766
  0.30953357  0.14806565 -0.01587007  0.06662974  0.03706932  0.17106722
 -0.03278566 -0.0334441   0.13855527 -0.05438197 -0.21337075 -0.04859404
  0.17125824 -0.06842338  0.0769766   0.09464139  0.07981446  0.18287898
  0.16078244 -0.15091591  0.0280329  -0.11844904  0.11621287  0.11389504
  0.22231635 -0.07441937 -0.17262302  0.03393755  0.3065542   0.03461281
  0.01399894 -0.01559456 -0.14189033 -0.44664936  0.11190843 -0.12791915
  0.0383955  -0.26294456 -0.08506965]


vector cosine distance                                   movie title
0                 0.000000                           American Pie (1999)
1                 0.072711                         American Pie 2 (2001)
2                 0.173660                              Road Trip (2000)
3                 0.211841           There's Something About Mary (1998)
4                 0.237007      American Wedding (American Pie 3) (2003)
5                 0.294478      American Reunion (American Pie 4) (2012)
6                 0.295322                       Wedding Crashers (2005)
7                 0.317829                       Meet the Parents (2000)
8                 0.324066                40-Year-Old Virgin, The (2005)
9                 0.338348                          Hangover, The (2009)
10                0.341572                               EuroTrip (2004)
11                0.343340                    Girl Next Door, The (2004)
12                0.368299                             Old School (2003)
13                0.368879                          Varsity Blues (1999)
14                0.375471                 Not Another Teen Movie (2001)
15                0.377030           Fast Times at Ridgemont High (1982)
16                0.379821                            Scary Movie (2000)
17                0.396807                  40 Days and 40 Nights (2002)
18                0.399863  Austin Powers: The Spy Who Shagged Me (1999)
19                0.405122          National Lampoon's Van Wilder (2002)

In [ ]:
get_top_similarities('Interstellar (2014)',model)

latent features  [ 6.44621655e-02 -8.44311672e-02  5.48277039e-02 -2.01214529e-02
 -1.24138994e-01 -1.44582024e-01 -6.31711823e-03 -3.26079411e-02
 -3.94971050e-02 -1.39955515e-01  1.28110421e-01  7.32449772e-02
 -2.00847257e-01  9.04600360e-02 -1.52989177e-02  7.92916962e-02
 -2.77226822e-02  5.03382973e-02 -4.50833140e-02  1.39105078e-01
 -8.73494154e-02 -5.99270537e-02  6.22861088e-02  1.66853580e-01
 -5.47999768e-02  4.84177282e-02  1.68920908e-01 -2.31056333e-03
  1.66054901e-01  3.93038165e-02 -1.09460781e-02 -1.06435361e-03
 -5.53337904e-02  5.94760441e-02  1.32218103e-01 -9.03621338e-02
 -2.32780951e-02  7.95579025e-02  5.97905194e-02  6.87805157e-02
  6.53859234e-02  5.81653323e-02  1.97808249e-02  1.03491806e-01
  9.91624222e-02  1.09394537e-01  1.89070367e-01 -1.17790783e-01
 -4.81439580e-02  9.19388633e-02 -4.19835368e-02 -9.41169984e-02
  7.81001775e-02  3.15089981e-02  4.63890287e-02 -3.15505868e-02
 -1.77265805e-02  1.76334303e-02  2.45053489e-02 -8.29957976e-02
  5.3610

vector cosine distance                                  movie title
0                 0.000000                          Interstellar (2014)
1                 0.182191                             Inception (2010)
2                 0.405793                           The Martian (2015)
3                 0.418184                               Arrival (2016)
4                 0.423212                        Shutter Island (2010)
5                 0.439319                         Prestige, The (2006)
6                 0.483326                Dark Knight Rises, The (2012)
7                 0.508976                               Gravity (2013)
8                 0.510178                      Dark Knight, The (2008)
9                 0.524504  Curious Case of Benjamin Button, The (2008)
10                0.527294                            Ex Machina (2015)
11                0.545581                      Edge of Tomorrow (2014)
12                0.555185                                Looper (2012)
13                0.561094                              Oblivion (2013)
14                0.565380         Black Mirror: White Christmas (2014)
15                0.569634                     Blade Runner 2049 (2017)
16                0.585266                    The Imitation Game (2014)
17                0.586304                             Gone Girl (2014)
18                0.594773                            Prometheus (2012)
19                0.601289                     Beautiful Mind, A (2001)

In [ ]:
get_top_similarities('Interstellar (2014)',model_51)

latent features  [ 0.03870975 -0.04911555 -0.04698225  0.17282322  0.0819401   0.04227637
 -0.09005075 -0.09429695 -0.0946198  -0.21090277  0.07556536 -0.26062582
 -0.03099011  0.04399217  0.18021129  0.06716032 -0.01004442 -0.1035076
  0.09459578 -0.09173456 -0.1427893   0.16913781 -0.13023862 -0.09959321
  0.23147992 -0.06186769 -0.18365104 -0.04388208 -0.06156603 -0.29268627
 -0.48783357  0.07776245 -0.00756313 -0.13938938 -0.26535143  0.05912309
 -0.06311937 -0.09811613 -0.16713551  0.05598517  0.11875746  0.04692571
  0.13526647 -0.03559879  0.16494618  0.0010975  -0.02319911 -0.07027163
  0.01538745  0.1443155  -0.11517608]


vector cosine distance                                        movie title
0                 0.000000                                Interstellar (2014)
1                 0.179947                                   Inception (2010)
2                 0.329174                                     Arrival (2016)
3                 0.344598                                 The Martian (2015)
4                 0.356825                              Shutter Island (2010)
5                 0.384199                                  Ex Machina (2015)
6                 0.392098                               Prestige, The (2006)
7                 0.429220                                   Gone Girl (2014)
8                 0.432126                                      Looper (2012)
9                 0.444609                                    Oblivion (2013)
10                0.446961               Black Mirror: White Christmas (2014)
11                0.448876                                   Prisoners (2013)
12                0.459364  Dragon Ball Z: Broly - The Legendary Super Sai...
13                0.459511                                  Disconnect (2012)
14                0.475844                                  Prometheus (2012)
15                0.476099                           Blade Runner 2049 (2017)
16                0.476795                                 The Endless (2017)
17                0.485540                          The Imitation Game (2014)
18                0.488822                                     Gravity (2013)
19                0.492553        Curious Case of Benjamin Button, The (2008)

In [ ]:
get_top_similarities()

In [ ]:
get_top_similarities('Goodfellas (1990)',model)

latent features  [ 0.05042252 -0.17604161 -0.08095945  0.10573576 -0.0159207   0.07242026
 -0.02466778 -0.27537637  0.09576249 -0.10338007  0.10905613 -0.01858406
 -0.06068555 -0.04215998 -0.05649722  0.07276177 -0.00109114 -0.0655873
  0.0663865  -0.0052256   0.15490044 -0.06457015 -0.04678435 -0.07277403
  0.11117491 -0.02477776 -0.10934335 -0.09312839 -0.11155446 -0.0272222
 -0.16173609 -0.00680988  0.02426091 -0.18150738 -0.01714965 -0.20032905
  0.08355983  0.0845439  -0.05993222 -0.11823399  0.1428361   0.03974157
  0.06509128 -0.13650596  0.02863261  0.08984095  0.04549644  0.09795518
 -0.0476615   0.12302478 -0.12963315 -0.08270353 -0.14173471 -0.11362095
 -0.07435524  0.11662564  0.06632037  0.08822267  0.20999396  0.14137636
  0.12469314  0.18299767 -0.10769337  0.00636934 -0.04188827  0.09479361
  0.05079023  0.24266888 -0.01625723 -0.13491245  0.0545388  -0.18838383
 -0.04094988 -0.02671469 -0.1226466  -0.08367018 -0.0490611  -0.00100096
 -0.09707435 -0.06740382  0.01338547

vector cosine distance                              movie title
0                 0.000000                        Goodfellas (1990)
1                 0.173207           Godfather: Part II, The (1974)
2                 0.185272                    Godfather, The (1972)
3                 0.257402                            Casino (1995)
4                 0.272526                          Scarface (1983)
5                 0.275206                       Raging Bull (1980)
6                 0.276021                    Reservoir Dogs (1992)
7                 0.302124                 Full Metal Jacket (1987)
8                 0.325087                       Taxi Driver (1976)
9                 0.339153                      Pulp Fiction (1994)
10                0.346346  The Godfather Trilogy: 1972-1990 (1992)
11                0.349740                     Donnie Brasco (1997)
12                0.356126                    Apocalypse Now (1979)
13                0.363863                     Departed, The (2006)
14                0.366479                           Platoon (1986)
15                0.390042            No Country for Old Men (2007)
16                0.392971                     Carlito's Way (1993)
17                0.395869                  Deer Hunter, The (1978)
18                0.435901                              Heat (1995)
19                0.446098                     Boogie Nights (1997)